In [1]:
import pandas as pd
import numpy as np
import collections, numpy
from numpy import genfromtxt
%matplotlib inline

In [2]:
data_names = ['A','G','CP','BP','CH','ECG','HR','EIA','HD']
data_name2index = {}
for i,val in enumerate(data_names):
    data_name2index[val] = i

In [3]:
def generate_CPT(target,all_data_frame,parents=[]):
    CPT = {}
    if len(parents)==0:
        #print target
        g = all_data_frame.groupby([target])
        #print g.groups
        normalizer = all_data_frame.shape[0]
        for target_value in g.groups.keys():
            #CPT[(target_value)] = lig.groups[target_value]
            CPT[(target_value,)] = g.groups[target_value].shape[0]*1.0/normalizer
        return CPT
    else:
        g = all_data_frame.groupby(parents)
        data_groups = g.groups.keys()
        #print g.groups
        for target_value in all_data_frame[target].unique():
            #print target_value
            for data_group in data_groups:
                data = all_data_frame.ix[list(g.groups[data_group])]
                cpt_key = list(data_group)
                cpt_key.insert(0,target_value)
                #print data[data[target]==target_value]
                normalizer = data.shape[0]
                #print normalizer
                #print "-->",data[data[target]==target_value].shape[0],"-->",normalizer
                CPT[tuple(cpt_key)] = data[data[target]==target_value].shape[0]*1.0/normalizer
        return CPT

In [4]:
CPT_table_names = ['CPT_A', 'CPT_G', 'CPT_CP_HD', 'CPT_BP_G', 'CPT_ECG_HD', 'CPT_HD_BP_CH_G', 'CPT_CH_G_A', 'CPT_EIA_HD', 'CPT_HR_A_HD']
def get_CPT_all(df):
    result = {}
    result['CPT_A'] = generate_CPT('A',df)
    result['CPT_G'] = generate_CPT('G',df)
    result['CPT_BP_G'] = generate_CPT('BP',df,['G'])
    result['CPT_CH_G_A'] = generate_CPT('CH',df,['G','A'])
    result['CPT_HD_BP_CH_G'] = generate_CPT('HD',df,['BP','CH','G'])
    result['CPT_CP_HD'] = generate_CPT('CP',df,['HD'])
    result['CPT_EIA_HD'] = generate_CPT('EIA',df,['HD'])
    result['CPT_ECG_HD'] = generate_CPT('ECG',df,['HD'])
    result['CPT_HR_A_HD'] = generate_CPT('HR',df,['A','HD'])
    result['CPT_HR_A_HD'] = generate_CPT('HR',df,['A','HD'])
    #print result.keys()
    return result

In [7]:
def calculate_probability_HD(value_dictionary,CPT_table):
    #print value_dictionary,'\n*******\n',CPT_table
    probability = CPT_table['CPT_HD_BP_CH_G'][(value_dictionary['HD'],value_dictionary['BP'],value_dictionary['CH'],value_dictionary['CH'])]\
                    *CPT_table['CPT_CP_HD'][(value_dictionary['CP'],value_dictionary['HD'])]\
                    *CPT_table['CPT_EIA_HD'][(value_dictionary['EIA'],value_dictionary['HD'])]\
                    *CPT_table['CPT_ECG_HD'][(value_dictionary['ECG'],value_dictionary['HD'])]\
                    *CPT_table['CPT_HR_A_HD'][(value_dictionary['HR'],value_dictionary['A'],value_dictionary['HD'])]
    return probability

In [8]:
import glob

files = {}
for i in range(5):
    files['data-train-'+str(i+1)+'.txt']='data-test-'+str(i+1)+'.txt'

accuracies = []
for train_file in files.keys():
    df_train = pd.read_csv('../Data/'+train_file, names=data_names,dtype="|S5")
    CPT_tables = get_CPT_all(df_train)
    df_test = pd.read_csv('../Data/'+files[train_file], names=data_names,dtype="|S5")
    target_HD = list(df_test['HD'])
    output_HD = []
    for index, row in df_test.iterrows():
        vals = {}
        for feature in data_names:
            vals[feature] = row[feature]
        vals['HD'] = '1'
        hd_1 = calculate_probability_HD(vals,CPT_tables)
        vals['HD'] = '2'
        hd_2 = calculate_probability_HD(vals,CPT_tables)
        p_hd_1 = hd_1/(hd_1+hd_2)
        p_hd_2 = hd_2/(hd_1+hd_2)
        if p_hd_1>p_hd_2:
            output_HD.append('1')
        else:
            output_HD.append('2')
    
    N = len(target_HD)
    matches = [i for i, j in zip(target_HD, output_HD) if i == j]
    print 'Accuracy of '+train_file+'-- >', len(matches)*1.0/N
    accuracies.append(len(matches)*1.0/N)
    print ''

print 'Mean accuracy --> ',np.mean(accuracies),' Standard deviation -->',np.std(accuracies)
        


Accuracy of data-train-4.txt-- > 0.766666666667

Accuracy of data-train-1.txt-- > 0.716666666667

Accuracy of data-train-3.txt-- > 0.666666666667

Accuracy of data-train-5.txt-- > 0.766666666667

Accuracy of data-train-2.txt-- > 0.816666666667

Mean accuracy -->  0.746666666667  Standard deviation --> 0.0509901951359


In [9]:
CPT_table_names = ['CPT_A', 'CPT_G', 'CPT_CP_HD', 'CPT_BP_G', 'CPT_ECG_HD', 'CPT_HD_BP_CH_A', 'CPT_CH_G_A', 'CPT_EIA_HD', 'CPT_HR_A_HD']
def get_CPT_all(df):
    result = {}
    result['CPT_A'] = generate_CPT('A',df)
    result['CPT_G'] = generate_CPT('G',df)
    result['CPT_BP_G'] = generate_CPT('BP',df,['G'])
    result['CPT_CH_G_A'] = generate_CPT('CH',df,['G','A'])
    result['CPT_HD_BP_CH_A'] = generate_CPT('HD',df,['BP','CH','A'])
    result['CPT_CP_HD'] = generate_CPT('CP',df,['HD'])
    result['CPT_EIA_HD'] = generate_CPT('EIA',df,['HD'])
    result['CPT_ECG_HD'] = generate_CPT('ECG',df,['HD'])
    result['CPT_HR_A_HD'] = generate_CPT('HR',df,['A','HD'])
    result['CPT_HR_A_HD'] = generate_CPT('HR',df,['A','HD'])
    #print result.keys()
    return result

In [10]:
def calculate_probability_HD(value_dictionary,CPT_table):
    #print value_dictionary,'\n*******\n',CPT_table
    probability = CPT_table['CPT_HD_BP_CH_A'][(value_dictionary['HD'],value_dictionary['BP'],value_dictionary['CH'],value_dictionary['CH'])]\
                    *CPT_table['CPT_CP_HD'][(value_dictionary['CP'],value_dictionary['HD'])]\
                    *CPT_table['CPT_EIA_HD'][(value_dictionary['EIA'],value_dictionary['HD'])]\
                    *CPT_table['CPT_ECG_HD'][(value_dictionary['ECG'],value_dictionary['HD'])]\
                    *CPT_table['CPT_HR_A_HD'][(value_dictionary['HR'],value_dictionary['A'],value_dictionary['HD'])]
    return probability

In [11]:
import glob

files = {}
for i in range(5):
    files['data-train-'+str(i+1)+'.txt']='data-test-'+str(i+1)+'.txt'

accuracies = []
for train_file in files.keys():
    df_train = pd.read_csv('../Data/'+train_file, names=data_names,dtype="|S5")
    CPT_tables = get_CPT_all(df_train)
    df_test = pd.read_csv('../Data/'+files[train_file], names=data_names,dtype="|S5")
    target_HD = list(df_test['HD'])
    output_HD = []
    for index, row in df_test.iterrows():
        vals = {}
        for feature in data_names:
            vals[feature] = row[feature]
        vals['HD'] = '1'
        hd_1 = calculate_probability_HD(vals,CPT_tables)
        vals['HD'] = '2'
        hd_2 = calculate_probability_HD(vals,CPT_tables)
        p_hd_1 = hd_1/(hd_1+hd_2)
        p_hd_2 = hd_2/(hd_1+hd_2)
        if p_hd_1>p_hd_2:
            output_HD.append('1')
        else:
            output_HD.append('2')
    
    N = len(target_HD)
    matches = [i for i, j in zip(target_HD, output_HD) if i == j]
    print 'Accuracy of '+train_file+'-- >', len(matches)*1.0/N
    accuracies.append(len(matches)*1.0/N)
    print ''

print 'Mean accuracy --> ',np.mean(accuracies),' Standard deviation -->',np.std(accuracies)
        


Accuracy of data-train-4.txt-- > 0.733333333333

Accuracy of data-train-1.txt-- > 0.7

Accuracy of data-train-3.txt-- > 0.616666666667

Accuracy of data-train-5.txt-- > 0.75

Accuracy of data-train-2.txt-- > 0.783333333333

Mean accuracy -->  0.716666666667  Standard deviation --> 0.0567646212198
